In [6]:
import pandas as pd
import os 
import numpy as np
import re
import openpyxl

In [7]:
# Directory paths for input and output
input_dir = 'C:\\Users\\DELL\\Downloads\\entry'
output_dir = 'C:\\Users\\DELL\\Downloads\\final'

# Name of the input file
file_entry_name = 'example_1.sql'

# Name of the desired output file
file_output_name = 'example_final.xlsx'

In [9]:
# Complete paths for the files
input_file_path = os.path.join(input_dir, file_entry_name)
output_file_path = os.path.join(output_dir, file_output_name)

# Check if the input file exists
if not os.path.exists(input_file_path):
    print(f"Error: Input file {input_file_path} not found.")
else:
    # Read the SQL file
    with open(input_file_path, 'r') as file:
        sql_query = file.read()

    # Split the query into parts based on SELECT statements
    select_sections = re.split(r'\bSELECT\b', sql_query, flags=re.IGNORECASE)

    # Patterns to match the required tables and fields
    pattern_tables = r'\b(FROM|JOIN|INNER JOIN|LEFT JOIN|RIGHT JOIN|FULL JOIN)\s+(MARKETINGOPERATIONS\.DM_CORE\.[\w\.]+|prod_datalake\.sfdc\.[\w\.]+|experience_index_db\.xi_operations_dev\.[\w\.]+|(?:\(\s*SELECT[^)]+\)))(?:\s+AS\s+(\w+)|\s+(\w+))?'
    pattern_fields = r'(\w+)\.(\w+)(?:\s+AS\s+(\w+))?'
    pattern_calculations = r'(SUM|COUNT|AVG|MIN|MAX|LOWER|UPPER|COALESCE|CASE|ROUND|CAST|CONCAT|SUBSTRING|REPLACE|DATEADD|DATEDIFF|ISNULL)\s*\(.*?\)'

    # Lists to store the extracted data
    table_data = []
    field_data = []

    # Process each select section
    for index, section in enumerate(select_sections):
        if not section.strip():
            continue

        # Find all table matches in the section
        table_matches = re.findall(pattern_tables, section, re.IGNORECASE)
        for match in table_matches:
            relationship = match[0]
            table_name = match[1]
            alias = match[2] if match[2] and match[2].lower() not in ["where", "group", "order", "by", "limit", "having"] else match[3] if match[3] and match[3].lower() not in ["where", "group", "order", "by", "limit", "having"] else ''
            table_data.append([f'SELECT {index + 1}', relationship, table_name, alias])

        # Find all field matches in the section
        field_matches = re.findall(pattern_fields, section, re.IGNORECASE)
        for match in field_matches:
            field_with_alias = f"{match[0]}.{match[1]} AS {match[2]}" if match[2] else f"{match[0]}.{match[1]}"
            calculation = ''
            if re.search(pattern_calculations, field_with_alias, re.IGNORECASE):
                calculation = re.search(pattern_calculations, field_with_alias, re.IGNORECASE).group(1)
            field_data.append([field_with_alias, match[0], match[1], match[2] if match[2] else '', 'YES' if calculation else 'NO', calculation])

    # Create DataFrames from the collected data
    df_tables = pd.DataFrame(table_data, columns=['Select Part', 'Relationship', 'Table Name', 'Alias']).drop_duplicates()
    df_fields = pd.DataFrame(field_data, columns=['Field', 'Prefix', 'Raw Field', 'Alias', 'Calculated', 'Calculation']).drop_duplicates()

    # Fill missing aliases in the tables
    df_tables['Alias'] = df_tables.apply(lambda row: row['Table Name'].split('.')[-1] if row['Alias'] == '' else row['Alias'], axis=1)

    # Count occurrences of SELECT, fields, tables, WHERE, and calculated